In [36]:
from glob import glob
from PIL import Image
import imagehash
from typing import List

def frame(fname: str) -> int:
    """for a given file name, return the frame index.
    
    for example, 'cropped/001.png' returns int 1.
    """
    idx = int(fname.split("/")[-1].split(".")[0])
    return idx


def remove_outliers(files: List[str]) -> List[str]:
    """remove images that are not similar with any other images in the batch.
    
    files is a sorted list of image file names, sorted by frame index number. this helps us
    keep files that are similar to its neighbors.

    we want to keep images that are similar with at least one other image in the batch.
    this is because we want to find long contiguous blocks of similar images, which would
    suggest static faces.
    """
    
    stack = []
    
    # remove other images

    for i in range(len(files) - 1):
        prev_f = files[i]
        next_f = files[i + 1]

        prev_h = imagehash.average_hash(Image.open(prev_f))
        next_h = imagehash.average_hash(Image.open(next_f))

        # keep if it's similar
        if next_h == prev_h:
            stack.append(prev_f)
            stack.append(next_f)
        else:
            pass
        
    return stack


def create_blocks(stack: List[str]) -> List[List[int]]:
    """create continguous blocks of similar frames.
    """
    MAX_FRAME_DIFF = 5

    blocks = []
    block = []

    for i in range(len(stack) - 1):
        p_idx = frame(stack[i])
        n_idx = frame(stack[i + 1])

        if (n_idx - p_idx) < MAX_FRAME_DIFF:
            block.append(p_idx)
            block.append(n_idx)
        else:
            blocks.append(block)
            block = []
            
    return blocks

In [84]:
def parse(folder: str, label: str, identifier: int):
    files = sorted(glob(folder + '/' + label + '/*'))
    
    files = list(filter(lambda x: int(x.split(".")[-2]) == identifier, files))

    if len(files) > 0:
        print("evaluating folder", folder)
        stack = remove_outliers(files)
        blocks = create_blocks(stack)
        for block in blocks:
            block = set(block)
            if len(block) > 5:
                print("block", identifier, sorted(block))
        print("-" * 50)

In [85]:
folders = glob("dataset/*")
folders

['dataset/547538174468711490516541559363']

In [86]:
for folder in folders:
    parse(folder, "not_static", 1)

evaluating folder dataset/547538174468711490516541559363
--------------------------------------------------
